In [162]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import random
from keras.models import load_model, Sequential, Model
from keras.layers import Cropping2D
import cv2
import os


In [163]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import cv2, numpy as np

def Model(weights_path=None, dropout=0.5, dropout_level=1, orig = True):
    if orig:
        model = Sequential()
        
        # Take 2x2 stride on the input to reduce dimensionality
        model.add(Lambda(lambda x: x[::2, ::2, :], input_shape=(160, 320, 3), output_shape=(80, 160, 3))) #(3, 80, 160)
        
        # Zero pad the width
        model.add(ZeroPadding2D(padding=(0, 20))) #(3, 80, 200)
        
        # Crop the height
        model.add(Cropping2D(cropping=((80-66, 0), (0, 0)))) #(3, 66, 200)
        
        # Normalize
        model.add(Lambda(lambda x: (x / 255.0) - 0.5, output_shape=(66, 200, 3)))
        
        # Original network
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Convolution2D(24, 5, 5, border_mode='valid', subsample=(2,2), activation='relu')) #(24, 31, 98)
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Convolution2D(36, 5, 5, border_mode='valid', subsample=(2,2), activation='relu')) #(36, 14, 47)
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Convolution2D(48, 5, 5, border_mode='valid', subsample=(2,2), activation='relu')) #(48, 5, 22)
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Convolution2D(64, 3, 3, border_mode='valid', subsample=(1,1), activation='relu')) #(64, 3, 20)
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Convolution2D(64, 3, 3, border_mode='valid', subsample=(1,1), activation='relu')) #(64, 1, 18)
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Flatten())
        model.add(Dense(100, activation='relu'))
        model.add(Dropout(dropout)) if dropout_level >= 1 else None
        model.add(Dense(50, activation='relu'))
        model.add(Dropout(dropout)) if dropout_level >= 1 else None
        model.add(Dense(10, activation='relu'))
        model.add(Dropout(dropout)) if dropout_level >= 1 else None
        model.add(Dense(1, activation='tanh'))
    else:
        model = Sequential()
        model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160, 320, 3), output_shape=(160, 320, 3)))
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Convolution2D(24, 5, 5, border_mode='valid', subsample=(3,3), activation='relu')) #(24, 52, 106)
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Convolution2D(36, 5, 5, border_mode='valid', subsample=(2,2), activation='relu')) #(36, 24, 51)
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Convolution2D(48, 5, 5, border_mode='valid', subsample=(2,2), activation='relu')) #(48, 10, 24)
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Convolution2D(64, 3, 3, border_mode='valid', subsample=(1,1), activation='relu')) #(64, 8, 22)
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Convolution2D(64, 3, 3, border_mode='valid', subsample=(1,1), activation='relu')) #(64, 6, 20)
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Flatten())
        model.add(Dense(100, activation='relu'))
        model.add(Dropout(dropout)) if dropout_level >= 1 else None
        model.add(Dense(50, activation='relu'))
        model.add(Dropout(dropout)) if dropout_level >= 1 else None
        model.add(Dense(10, activation='relu'))
        model.add(Dropout(dropout)) if dropout_level >= 1 else None
        model.add(Dense(1, activation='tanh'))

    if weights_path:
        model.load_weights(weights_path, by_name=True)

    return model

In [164]:
track1_dir = '/home/carnd/Dropbox/udacity-data/track1'

folders_to_exclude = ['trial1_offroad_fwd', 'trial1_offroad_bwd']
track1_data_dirs = [x for x in os.listdir(track1_dir) if x not in folders_to_exclude]
print(track1_data_dirs)
track1_data_dirs = [track1_dir + '/' + x for x in track1_data_dirs]


driving_log_df = None

for data_dir in track1_data_dirs:
    df = pd.read_csv(data_dir + "/driving_log.csv", header=None, names=["center","left","right","steering","throttle","brake","speed"])

    cols = ['center', 'left', 'right']
    for col in cols:
        df[col] = df[col].str.strip()
        df[col] = df[col].str.split("/").apply(lambda x: x[-1])
    df[['center', 'left', 'right']] = data_dir + "/IMG/" + df[['center', 'left', 'right']]
#    "/home/carnd/Dropbox/udacity-data/track1/trial1_offroad_fwd//Users/macbook/Development/personal/udacity-car/CarND-Behavioral-Cloning-P3/trial1_offroad_fwd/IMG/center_2017_02_19_10_05_58_231.jpg"
    
    if driving_log_df is None:
        driving_log_df = df
    else:
        driving_log_df = pd.concat([driving_log_df, df])

# # shuffle
# np.random.seed(42)
# driving_log_df = driving_log_df.reindex(np.random.permutation(driving_log_df.index), copy=False)

driving_log_df.head()

['trial2', 'trial2b', 'trial1_recovery_fwd', 'trial1_corners_bwd', 'trial2-corner', 'trial1_recovery_bwd', 'trial1_corners_fwd']


,center,left,right,steering,throttle,brake,speed
0,/home/carnd/Dropbox/udacity-data/track1/trial2...,/home/carnd/Dropbox/udacity-data/track1/trial2...,/home/carnd/Dropbox/udacity-data/track1/trial2...,-0.132702,1.0,0.0,30.19033
1,/home/carnd/Dropbox/udacity-data/track1/trial2...,/home/carnd/Dropbox/udacity-data/track1/trial2...,/home/carnd/Dropbox/udacity-data/track1/trial2...,-0.130417,1.0,0.0,30.19027
2,/home/carnd/Dropbox/udacity-data/track1/trial2...,/home/carnd/Dropbox/udacity-data/track1/trial2...,/home/carnd/Dropbox/udacity-data/track1/trial2...,-0.108822,1.0,0.0,30.19071
3,/home/carnd/Dropbox/udacity-data/track1/trial2...,/home/carnd/Dropbox/udacity-data/track1/trial2...,/home/carnd/Dropbox/udacity-data/track1/trial2...,-0.108822,1.0,0.0,30.19054
4,/home/carnd/Dropbox/udacity-data/track1/trial2...,/home/carnd/Dropbox/udacity-data/track1/trial2...,/home/carnd/Dropbox/udacity-data/track1/trial2...,-0.078014,1.0,0.0,30.19083


In [165]:
def get_next_image_generator(df, position = 'center', offset = 0.2):
    for idx, image_path in enumerate(df[position]):
        img = cv2.imread(image_path)
        yield img


In [166]:
tempgen = get_next_image_generator(driving_log_df)
sample = next(tempgen)
print("Dimension of image: H x W X D = ", sample.shape)
print("# of images: ", len(driving_log_df))

print("Steering range: Min=", np.min(driving_log_df['steering']), " , Max=", np.max(driving_log_df['steering']))
print("Throttle range: Min=", np.min(driving_log_df['throttle']), " , Max=", np.max(driving_log_df['throttle']))
print("Brake range: Min=", np.min(driving_log_df['brake']), " , Max=", np.max(driving_log_df['brake']))
print("Speed range: Min=", np.min(driving_log_df['speed']), " , Max=", np.max(driving_log_df['speed']))

print("image Min: ", np.min(sample))
print("image Max: ", np.max(sample))
sample

Dimension of image: H x W X D =  (160, 320, 3)
# of images:  10025
Steering range: Min= -1.0  , Max= 0.9576946
Throttle range: Min= 0.0  , Max= 1.0
Brake range: Min= 0.0  , Max= 1.0
Speed range: Min= 8.622601e-05  , Max= 30.6053
image Min:  0
image Max:  255


array([[[230, 177, 144],
        [237, 184, 151],
        [230, 177, 144],
        ..., 
        [173, 134, 106],
        [173, 134, 106],
        [173, 134, 106]],

       [[231, 178, 145],
        [252, 199, 166],
        [244, 191, 158],
        ..., 
        [173, 134, 106],
        [173, 134, 106],
        [173, 134, 106]],

       [[247, 194, 161],
        [226, 173, 140],
        [230, 177, 144],
        ..., 
        [174, 135, 107],
        [174, 135, 107],
        [174, 135, 107]],

       ..., 
       [[129, 152, 154],
        [127, 150, 152],
        [129, 152, 154],
        ..., 
        [117, 131, 130],
        [128, 142, 141],
        [142, 156, 155]],

       [[122, 145, 147],
        [130, 153, 155],
        [141, 164, 166],
        ..., 
        [112, 126, 125],
        [120, 134, 133],
        [129, 143, 142]],

       [[124, 147, 149],
        [125, 148, 150],
        [134, 157, 159],
        ..., 
        [121, 135, 134],
        [116, 130, 129],
        [110, 124,

In [167]:
def get_next_feature(df, batch_size = 10, mode = 'train', position = 'center', offset = 0.1, val_portion = 0.2, include_mirror=True):
    df['mirror'] = False
    if include_mirror:
        dfMirror = df.copy()
        dfMirror['mirror'] = True
        dfMirror['steering'] = -dfMirror['steering']
        df = pd.concat([df, dfMirror])
        
    if position == 'all':
        dfLeft = df.copy()
        dfLeft['target'] = 'left'
        dfCenter = df.copy()
        dfCenter['target'] = 'center'
        dfRight = df.copy()
        dfRight['target'] = 'right'
        df = pd.concat([dfLeft, dfCenter, dfRight])
    else:
        df['target'] = position
        
    #Shuffle
    df = df.sample(frac=1).reset_index(drop=True)
    
    total_len = len(df)
    val_len = int(val_portion * total_len)
    train_len = total_len - val_len
    
    if mode == "train":
        df = df[:train_len]
    else:
        df = df[train_len:]
    
    row = df.iloc[0]
    sample_image = cv2.imread(row[row['target']])

    image_size = sample_image.shape

    inputs = np.zeros([batch_size, *image_size]) #length of prediction output
    targets = np.zeros([batch_size])
    
    count = 0
    
    while(True):
        for idx in range(len(df)):
            row = df.iloc[idx]
            image_path = row[row['target']]
            img = cv2.imread(image_path)
            img = img[np.newaxis, :, :, :]

            if row['mirror']:
                img = img[:,::-1,:]

            inputs[count] = img
            targets[count] = df.iloc[idx]['steering']
            if row['target'] == 'right':
                targets[count] -= offset
            elif row['target'] == 'left':
                targets[count] += offset

            count += 1
            if count == batch_size:
                yield inputs, targets
                inputs = np.zeros([batch_size, *image_size])
                targets = np.zeros([batch_size])
                count = 0
                


In [168]:
# Define the model

#model = Model() # The nVidia model
model = Model(dropout=0.5, dropout_level=0, orig = False)

In [169]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=["accuracy"])

# train model
EPOCHS = 5
BATCH_SIZE = 300
OFFSET = 0.2 #0.034 radians is 2 degrees
VAL_PORTION = 0.2
INCLUDE_MIRROR = True


# # Train on the center data
# position = 'center'
# print(position)
# train_generator_center = get_next_feature(driving_log_df, 10, 'train', position, OFFSET, VAL_PORTION, INCLUDE_MIRROR)
# validation_generator_center = get_next_feature(driving_log_df, 10, 'val', position, OFFSET, VAL_PORTION, INCLUDE_MIRROR)
# model.fit_generator(train_generator_center, BATCH_SIZE, EPOCHS, verbose=2, validation_data=validation_generator_center, nb_val_samples=BATCH_SIZE/3)

# # Train on the left camera data
# position = 'left'
# print(position)
# train_generator_left = get_next_feature(driving_log_df, 10, 'train', position, OFFSET, VAL_PORTION, INCLUDE_MIRROR)
# validation_generator_left = get_next_feature(driving_log_df, 10, 'val', position, OFFSET, VAL_PORTION, INCLUDE_MIRROR)
# model.fit_generator(train_generator_left, BATCH_SIZE, EPOCHS, verbose=2, validation_data=validation_generator_left, nb_val_samples=BATCH_SIZE/3)

# # Train on the right camera data
# position = 'right'
# print(position)
# train_generator_right = get_next_feature(driving_log_df, 10, 'train', position, OFFSET, VAL_PORTION, INCLUDE_MIRROR)
# validation_generator_right = get_next_feature(driving_log_df, 10, 'val', position, OFFSET, VAL_PORTION, INCLUDE_MIRROR)
# model.fit_generator(train_generator_right, BATCH_SIZE, EPOCHS, verbose=2, validation_data=validation_generator_right, nb_val_samples=BATCH_SIZE/3)

# Train on all the data
position = 'all'
train_generator_all = get_next_feature(driving_log_df, 10, 'train', position, OFFSET, VAL_PORTION, INCLUDE_MIRROR)
validation_generator_all = get_next_feature(driving_log_df, 10, 'val', position, OFFSET, VAL_PORTION, INCLUDE_MIRROR)
model.fit_generator(train_generator_all, BATCH_SIZE, EPOCHS, verbose=2, validation_data=validation_generator_all, nb_val_samples=BATCH_SIZE/3)

# driving_log_df



Epoch 1/5
2s - loss: 0.0383 - acc: 0.0033 - val_loss: 0.0493 - val_acc: 0.0000e+00
Epoch 2/5
1s - loss: 0.0406 - acc: 0.0033 - val_loss: 0.0414 - val_acc: 0.0000e+00
Epoch 3/5
1s - loss: 0.0418 - acc: 0.0067 - val_loss: 0.0350 - val_acc: 0.0100
Epoch 4/5
1s - loss: 0.0396 - acc: 0.0033 - val_loss: 0.0379 - val_acc: 0.0200
Epoch 5/5
1s - loss: 0.0387 - acc: 0.0000e+00 - val_loss: 0.0374 - val_acc: 0.0000e+00


In [170]:
model.save('model.h5')

# # returns a compiled model
# # identical to the previous one
# model = load_model('model.h5')

In [171]:
# Test to determine offset parameter
for offset in range(0, 100, 5):
    offset = offset * 0.01
    print("Offset = ", offset)
    
    EPOCHS=5
    train_gen = get_next_feature(driving_log_df, 10, 'train', 'right', OFFSET, VAL_PORTION, INCLUDE_MIRROR)
    val_gen = get_next_feature(driving_log_df, 10, 'val', 'center', offset, VAL_PORTION, INCLUDE_MIRROR)

    # want to produce center training data, then test on 
    model.fit_generator(train_gen, BATCH_SIZE, EPOCHS, verbose=2, validation_data=val_gen, nb_val_samples=BATCH_SIZE/3)


Offset =  0.0
Epoch 1/5
1s - loss: 0.0252 - acc: 0.0000e+00 - val_loss: 0.0520 - val_acc: 0.0100
Epoch 2/5
1s - loss: 0.0138 - acc: 0.0000e+00 - val_loss: 0.0330 - val_acc: 0.0100
Epoch 3/5
1s - loss: 0.0140 - acc: 0.0000e+00 - val_loss: 0.0577 - val_acc: 0.0100
Epoch 4/5
1s - loss: 0.0156 - acc: 0.0000e+00 - val_loss: 0.0916 - val_acc: 0.0200
Epoch 5/5
1s - loss: 0.0154 - acc: 0.0000e+00 - val_loss: 0.0400 - val_acc: 0.0100
Offset =  0.05
Epoch 1/5
1s - loss: 0.0114 - acc: 0.0033 - val_loss: 0.0676 - val_acc: 0.0100
Epoch 2/5
1s - loss: 0.0194 - acc: 0.0000e+00 - val_loss: 0.0567 - val_acc: 0.0000e+00
Epoch 3/5
1s - loss: 0.0132 - acc: 0.0000e+00 - val_loss: 0.0626 - val_acc: 0.0000e+00
Epoch 4/5
1s - loss: 0.0112 - acc: 0.0000e+00 - val_loss: 0.0344 - val_acc: 0.0000e+00
Epoch 5/5
1s - loss: 0.0181 - acc: 0.0000e+00 - val_loss: 0.0684 - val_acc: 0.0100
Offset =  0.1
Epoch 1/5
1s - loss: 0.0209 - acc: 0.0000e+00 - val_loss: 0.0467 - val_acc: 0.0100
Epoch 2/5
1s - loss: 0.0106 - acc: 0